In [2]:
from U2GNN_pytorch.util import S2VGraph
import networkx as nx
import numpy as np

In [3]:
def load_data(path,dataset, degree_as_tag):
    '''
        dataset: name of dataset
        test_proportion: ratio of test train split
        seed: random seed for random splitting of dataset
    '''

    print('loading data')
    g_list = []
    label_dict = {}
    feat_dict = {}

    with open('%s/dataset/%s/%s.txt' % (path,dataset, dataset), 'r') as f:
        n_g = int(f.readline().strip())
        for i in range(n_g):
            row = f.readline().strip().split()
            n, l = [int(w) for w in row]
            if not l in label_dict:
                mapped = len(label_dict)
                label_dict[l] = mapped
            g = nx.Graph()
            node_tags = []
            node_features = []
            n_edges = 0
            for j in range(n):
                g.add_node(j)
                row = f.readline().strip().split()
                tmp = int(row[1]) + 2
                if tmp == len(row):
                    # no node attributes
                    row = [int(w) for w in row]
                    attr = None
                else:
                    row, attr = [int(w) for w in row[:tmp]], np.array([float(w) for w in row[tmp:]])
                if not row[0] in feat_dict:
                    mapped = len(feat_dict)
                    feat_dict[row[0]] = mapped
                node_tags.append(feat_dict[row[0]])

                if tmp > len(row):
                    node_features.append(attr)

                n_edges += row[1]
                for k in range(2, len(row)):
                    g.add_edge(j, row[k])

            if node_features != []:
                node_features = np.stack(node_features)
                node_feature_flag = True
            else:
                node_features = None
                node_feature_flag = False

            assert len(g) == n

            g_list.append(S2VGraph(g, l, node_tags))
    #return g_list, len(label_dict), node_features
    
    #add labels and edge_mat       
    for g in g_list:
        g.neighbors = [[] for i in range(len(g.g))]
        for i, j in g.g.edges():
            g.neighbors[i].append(j)
            g.neighbors[j].append(i)
        degree_list = []
        for i in range(len(g.g)):
            g.neighbors[i] = g.neighbors[i]
            degree_list.append(len(g.neighbors[i]))
        g.max_neighbor = max(degree_list)

        g.label = label_dict[g.label]

        edges = [list(pair) for pair in g.g.edges()]
        edges.extend([[i, j] for j, i in edges])

        deg_list = list(dict(g.g.degree(range(len(g.g)))).values())

        g.edge_mat = np.transpose(np.array(edges, dtype=np.int32), (1,0))

    if degree_as_tag:
        for g in g_list:
            g.node_tags = list(dict(g.g.degree).values())

    #Extracting unique tag labels   
    tagset = set([])
    for g in g_list:
        tagset = tagset.union(set(g.node_tags))

    tagset = list(tagset)
    tag2index = {tagset[i]:i for i in range(len(tagset))}

    for g in g_list:
        g.node_features = np.zeros((len(g.node_tags), len(tagset)), dtype=np.float32)
        g.node_features[range(len(g.node_tags)), [tag2index[tag] for tag in g.node_tags]] = 1


    print('# classes: %d' % len(label_dict))
    print('# maximum node tag: %d' % len(tagset))

    print("# data: %d" % len(g_list))

    return g_list, len(label_dict)
    

In [4]:
list_g,num = load_data("/home/keshav/courses/master_thesis/original_graph_transformer/Graph-Transformer",'PROTEINS',False)

loading data
# classes: 2
# maximum node tag: 3
# data: 1113


In [5]:
print(list_g[0].g.nodes())
print(list_g[0].node_tags)
print(list_g[0].node_features.shape)

[0, 11, 22, 32, 1, 23, 31, 41, 2, 24, 29, 34, 3, 4, 28, 38, 39, 14, 5, 6, 7, 26, 8, 19, 25, 9, 10, 35, 21, 20, 12, 33, 13, 15, 16, 17, 18, 36, 30, 40, 27, 37]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(42, 3)


In [6]:
print(list_g[1].g.nodes())
print(list_g[1].node_tags)
print(list_g[1].node_features.shape)

[0, 1, 2, 3, 4, 5, 6, 14, 7, 8, 15, 9, 10, 18, 11, 12, 19, 17, 13, 16, 20, 24, 22, 21, 23, 25, 26]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(27, 3)


In [7]:
print(list_g[2].g.nodes())
print(type(list_g[2].node_tags))
print(list_g[2].node_features.shape)

[0, 1, 2, 9, 3, 4, 5, 6, 7, 8]
<class 'list'>
(10, 3)


In [8]:
import itertools
from sklearn.neighbors import kneighbors_graph
from collections import Counter
def build_kneighbors(X, knn=True, n_neighbors=20):
    if knn:
        A = kneighbors_graph(X, n_neighbors, include_self=True)
        A = np.array(A.todense())
        A = (A + A.T)/2
        A = (A >0).astype(int)
    else:
        A = pairwise_kernels(X, metric='rbf', gamma=1)
    return A

In [9]:
def add_multiple_layers(list_g,n_top_attrs = 2,knn_featrs = True,num_knn = 3):
    for g in list_g:
        g.knn_g = None
        g.attr_gs = []
        if(knn_featrs):
            A = build_kneighbors(g.node_features,n_neighbors = num_knn)
            g_knn = nx.convert_matrix.from_numpy_matrix(A)
            g.knn_g = g_knn
        if(n_top_attrs>0):
            c = Counter(g.node_tags)
            top_vals = c.most_common(n_top_attrs)
            for value, count in top_vals:
                G_cur = nx.Graph()
                G_cur.add_nodes_from(g.g.nodes())
                imp_nodes = [node for i,node in enumerate(g.g.nodes()) if (g.node_tags[i] == value)]

                for u,v in itertools.combinations(imp_nodes, 2):
                    G_cur.add_edge(u,v)
                g.attr_gs.append(G_cur)
                
        
add_multiple_layers(list_g, 1)

In [10]:
print(list_g[2].attr_gs)

In [11]:
list_g[2].knn_g

In [12]:
list_g[4].edge_mat

array([[ 0,  0,  0,  0,  4,  4,  5,  9,  9, 10,  1,  1,  1,  1,  1,  2,
         2,  3,  6,  6,  7,  4,  5,  9, 10,  5, 10, 10,  7, 10,  7,  2,
         3,  6,  7,  8,  3,  6,  6,  7,  8,  8],
       [ 4,  5,  9, 10,  5, 10, 10,  7, 10,  7,  2,  3,  6,  7,  8,  3,
         6,  6,  7,  8,  8,  0,  0,  0,  0,  4,  4,  5,  9,  9, 10,  1,
         1,  1,  1,  1,  2,  2,  3,  6,  6,  7]], dtype=int32)

In [15]:
import torch
device = "cpu"
def get_graphpool(batch_graph):
    start_idx = [0]
    # compute the padded neighbor list
    for i, graph in enumerate(batch_graph):
        start_idx.append(start_idx[i] + len(graph.g))

    idx = []
    elem = []
    for i, graph in enumerate(batch_graph):
        elem.extend([1] * len(graph.g))
        idx.extend([[i, j] for j in range(start_idx[i], start_idx[i + 1], 1)])

    elem = torch.FloatTensor(elem)
    idx = torch.LongTensor(idx).transpose(0, 1)
    graph_pool = torch.sparse.FloatTensor(idx, elem, torch.Size([len(batch_graph), start_idx[-1]]))

    return graph_pool.to(device)

In [16]:
val = get_graphpool(list_g)

In [22]:
torch.zeros((10,10))[0,:].shape

torch.Size([10])